In [1]:
!pip install ultralytics roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 122.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import cv2
import time
import os
import json
from ultralytics import YOLO

In [3]:
def measure_performance(model_name, model_path, video_path):
    model = YOLO(model_path)
    video = cv2.VideoCapture(video_path)

    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(f'res_{model_name}.mp4', fourcc, 30, (width, height))

    total_inference_time = 0
    inference_time = 0
    encoding_time = 0
    frame_count = 0

    while video.isOpened():
        start_total = time.perf_counter() # 전체 시작

        ret, frame = video.read()
        if not ret:
            break

        start_inf = time.perf_counter() # 추론 시작
        results = model(frame, verbose=False, device=0) # Colab GPU
        end_inf = time.perf_counter() # 추론 끝

        annotated_frame = results[0].plot() # 결과 그림 그리기

        start_enc = time.perf_counter() # 인코딩 시작
        out.write(annotated_frame)
        end_enc = time.perf_counter() # 인코딩 끝

        end_total = time.perf_counter() # 전체 끝

        total_inference_time += (end_total - start_total)
        inference_time += (end_inf - start_inf)
        encoding_time += (end_enc - start_enc)
        frame_count += 1

    video.release()
    out.release()

    total_fps = frame_count / total_inference_time
    inf_fps = frame_count / inference_time
    enc_fps = frame_count / encoding_time

    return total_fps, inf_fps, enc_fps

In [4]:
BASE_DIR = '/content/drive/MyDrive/V_Marker/'

video_path = os.path.join(BASE_DIR, "test.mp4")
models = {
    "yolo11_nano": os.path.join(BASE_DIR, "V_marker_detection", "runs", "detect", "yolo11n_Inair", "weights", "best.pt"),
    "yolo11_small": os.path.join(BASE_DIR, "V_marker_detection", "runs", "detect", "yolo11s_Inair", "weights", "best.pt")
}

results = {}

for model, path in models.items():
    total_fps, inf_fps, enc_fps = measure_performance(model, path, video_path=video_path)
    results[model] = f"total_fps: {total_fps}, inf_fps: {inf_fps}, enc_fps: {enc_fps}"

print(json.dumps(results, indent=4))

{
    "yolo11_nano": "total_fps: 13.067083501439324, inf_fps: 51.83964425064237, enc_fps: 23.057100067877684",
    "yolo11_small": "total_fps: 12.990490665868203, inf_fps: 52.53720663212546, enc_fps: 23.35579668794629"
}
